In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1, num_layers=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Initialize hidden state with zeros (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0) ,self.hidden_size).to(x.device)
        
        # Forward pass through RNN
        out, _ = self.rnn(x, h0)
        
        # Take output from the last time step (many-to-one)
        out = self.fc(out[:, -1, :])
        return out


# Parameters
seq_length = 10
num_samples = 1000
hidden_size = 50
num_layers = 1
num_epochs = 100
learning_rate = 0.001



df = pd.read_csv('train_data.csv',index_col=0,parse_dates=True)
seq_length = df.shape[1]-1
X = df.iloc[:,:-1].values
X = X.reshape(-1, seq_length, 1)

y = df.iloc[:,-1].values

# Convert to PyTorch tensors
X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.float32)

# Initialize model, define loss and optimizer
model = RNN(input_size=12, hidden_size=hidden_size, output_size=1, num_layers=num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    test_output = model(X_train).detach().numpy()

# Plot predictions vs actual values
plt.plot(y_train, label='Actual data')
plt.plot(test_output, label='Predicted data')
plt.legend()
plt.show()


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

In [ ]:
y_train

tensor([[ 0.0000e+00],
        [ 6.5270e-01],
        [ 1.0000e+00],
        ...,
        [-1.0000e+00],
        [-6.5270e-01],
        [-2.4871e-16]])